In [73]:
import pandas as pd
import numpy as np
import re
import networkx as nx
import matplotlib.pyplot as plt

In [74]:
from collections import Counter

In [103]:
magazine_s = pd.read_csv('./magazine/final/magazine_Sl.csv', sep='|', encoding = 'utf-8')

In [104]:
magazine_s.head()

,magazine,brand,category,title,link,date,contents,title_brand,check,word_laurent,sentences_tag,ADFN,item
0,A,YSL,CELEBRITY,"깊게 뿌리내리기 위하여, 수영",http://www.allurekorea.com/2020/11/20/%ea%b9%8...,2020-11-20,['누구보다 본인에게 엄격하지만 수용할 줄 아는 너른 마음도 지녔다. 오늘도 부단히...,0,0,"['빨간 더블 브레스티드 코트, 타이츠, 라텍스 소재 보디슈트와 사이하이 부츠는 모...","[('[', 'SS'), (""'"", 'SS'), ('빨갛', 'VA'), ('ㄴ',...","['빨갛', '더블', '코트', '타이츠', '라텍스', '소재', '보디', '...","['코트', '슈트', '부츠', 'LAURENT', 'LAURENT']"
1,A,YSL,FASHION,코트 체크,http://www.allurekorea.com/2020/11/10/%ec%bd%9...,2020-10-26,"['겨울이 오기 전 기억해야 할 코트의 면면.', '90’S STANDARD', '...",0,0,"['클래식한 캐멀 컬러 코트는 1백78만원, 우영미(Wooyoungmi). 버건디 ...","[('[', 'SS'), (""'"", 'SS'), ('클래식', 'NNG'), ('하...","['클래식', '컬러', '코트', '우영', 'WOOYOUNGMI', '버건디',...","['코트', '셔츠', '스커트', 'LAURENT', 'LAURENT', '코트'..."
2,A,YSL,FASHION,데님이여 영원하라,http://www.allurekorea.com/2020/11/06/%eb%8d%b...,2020-10-26,['데님 팬츠를 선택할 때 가장 중요한 것은 자신의 체형에 맞는 핏을 고르는 것이다...,0,0,"[' 짙은 그레이 컬러의 데님 팬츠는 95만5천원, 생로랑 바이 안토니 바카렐로(...","[('[', 'SS'), (""'"", 'SS'), ('짙', 'VV'), ('은', ...","['짙', '그레이', '컬러', '데님', '팬츠', 'LAURENT', 'ANT...","['데님', '팬츠', 'LAURENT', 'LAURENT']"
3,A,YSL,FASHION,안경 하세요?!,http://www.allurekorea.com/2020/10/29/%ec%95%8...,2020-10-23,['모든 게 익고 마는 금빛 가을에는 탐스러운 금테 안경을. 전체 메탈로 장식한 다...,0,0,"[' YSL 모노그램을 새긴 블랙 에나멜 안경다리의 둥근 금테 안경은 40만원대,...","[('[', 'SS'), (""'"", 'SS'), ('YSL', 'OL'), ('모노...","['YSL', '모노그램', '새기', '블랙', '에나멜', '안경', '다리',...","['LAURENT', 'LAURENT']"
4,A,YSL,FASHION,"가을의 정석, 트렌치 코트 입는 법 8가지",http://www.allurekorea.com/2020/10/11/%ea%b0%8...,2020-09-27,"['정직하고 말끔하게, 혹은 자유롭고 분방하게. 변화무쌍한 계절처럼 다양한 모습을 ...",0,0,"['컬러블록 트렌치코트는 가격미정, 르쥬(Leje). 컵케이크 프린트 티셔츠는 27...","[('[', 'SS'), (""'"", 'SS'), ('컬러', 'NNG'), ('블록...","['컬러', '블록', '트렌치', '코트', 'LEJE', '컵', '케이크', ...","['코트', '티셔츠', '데님', '팬츠', '벨트', '슈즈', '선글라스', ..."


In [105]:
len(magazine_s)

1274

In [106]:
mag_s = magazine_s[['magazine', 'brand','title', 'link', 'date', 'word_laurent','ADFN']]

In [107]:
mag_s.head()

,magazine,brand,title,link,date,word_laurent,ADFN
0,A,YSL,"깊게 뿌리내리기 위하여, 수영",http://www.allurekorea.com/2020/11/20/%ea%b9%8...,2020-11-20,"['빨간 더블 브레스티드 코트, 타이츠, 라텍스 소재 보디슈트와 사이하이 부츠는 모...","['빨갛', '더블', '코트', '타이츠', '라텍스', '소재', '보디', '..."
1,A,YSL,코트 체크,http://www.allurekorea.com/2020/11/10/%ec%bd%9...,2020-10-26,"['클래식한 캐멀 컬러 코트는 1백78만원, 우영미(Wooyoungmi). 버건디 ...","['클래식', '컬러', '코트', '우영', 'WOOYOUNGMI', '버건디',..."
2,A,YSL,데님이여 영원하라,http://www.allurekorea.com/2020/11/06/%eb%8d%b...,2020-10-26,"[' 짙은 그레이 컬러의 데님 팬츠는 95만5천원, 생로랑 바이 안토니 바카렐로(...","['짙', '그레이', '컬러', '데님', '팬츠', 'LAURENT', 'ANT..."
3,A,YSL,안경 하세요?!,http://www.allurekorea.com/2020/10/29/%ec%95%8...,2020-10-23,"[' YSL 모노그램을 새긴 블랙 에나멜 안경다리의 둥근 금테 안경은 40만원대,...","['YSL', '모노그램', '새기', '블랙', '에나멜', '안경', '다리',..."
4,A,YSL,"가을의 정석, 트렌치 코트 입는 법 8가지",http://www.allurekorea.com/2020/10/11/%ea%b0%8...,2020-09-27,"['컬러블록 트렌치코트는 가격미정, 르쥬(Leje). 컵케이크 프린트 티셔츠는 27...","['컬러', '블록', '트렌치', '코트', 'LEJE', '컵', '케이크', ..."


In [108]:
mag_s.loc[0, 'ADFN'][2]

'빨'

In [109]:
mag_s['ADFN_2'] = mag_s['ADFN']

C:\Users\wnsgo\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [110]:
len(mag_s)

1274

In [111]:
mag_s.to_csv('./magazine/final/magazine_SaintLaurent.csv', sep = '|', encoding = 'utf-8')

### 쪼개진 단어 붙이기

#### 사전에 등록했으면 하는 단어
['레디 투 웨어', '공방', '트위드', 'FW', 'F/W', 'SS', 'S/S', '오간자', '아름다움', '그랑 팔레', '캐주얼', '런웨이', 
 '오프숄더', '레이어드', '인스타그램', '스타일링']

In [112]:
for word in mag_s['ADFN_2']:
    if '분위기' in word:
        print(word)
        print('-'*50)

['시즌', '아이템', '기도', '낙', '낙한', '핏', '재킷', '팬츠', '활동적', '이면서', '여유', '휴양지', '완성', '자크', 'ALEXANDER', '왕', '컬러', '한', '조', '팬츠', 'SALVATORE', 'FERRAGAMO', '코쿤', '숄더', '재킷', '스커트', '스카프', '챙', '넓', '모자', '눈여겨보', '끌', '느낌', '밑단', '슬리브', '롤', '업', '끌', '부츠', '웨스턴', '분위기', '디테일', '관심', '가지', '막스', '마라', '반', '셔츠', 'LAURENT', '베스트', '스타일', '더위', '힘겹', '구현', '스타일', 'MICHAEL', 'KORS', '컬렉션', '오프', '화이트', '테일러', '재킷', '미니', '원피스', '여름', '밤', '파티', '손색', '버뮤다', '데님', '팬츠', '터번', '두르', '마린', '풍', '재킷', 'LAURENT', '히피', '스타일', '실', '테일러', '코트', '올리비', '란제리', '응용']
--------------------------------------------------
['시즌', '디스코', '후예', '자리', '디스코', '와이드', '칼라', '남', '갈', '어깨선', '웃돌', '넓', '퍼지', '칼라', '코트', '재킷', '셔츠', '아이템', '가리', '리듬감', '부여', '칼라', '머스터드', '컬러', '장식', '투', '톤', '코트', '대표적', '답', '실루엣', '커팅', '부드럽', '컬러', '조합', '신경', '느낌', '세일러', '와이드', '칼라', '재킷', '트랙', '슈트', 'SALVATORE', 'FERRAGAMO', '벽돌', '컬러', '셔츠', '톤', '톤', '재킷', '터틀넥', '빅토리아', '컴', '마린', '풍', '의상', '와이드', '칼라', 'LAURENT', '투',

In [113]:
# 테일러 / 레이어
mag_s['ADFN_2'] = mag_s['ADFN_2'].str.replace("'테일러', '드'", "테일러드")
mag_s['ADFN_2'] = mag_s['ADFN_2'].str.replace("'레이어', '드'", "레이어드")

mag_s['ADFN_2'] = mag_s['ADFN_2'].str.replace("'테일러', '링'", "테일러링")
mag_s['ADFN_2'] = mag_s['ADFN_2'].str.replace("'레이어', '링'", "레이어링")

mag_s['ADFN_2'] = mag_s['ADFN_2'].str.replace("'에스', '닉'", "에스닉")

C:\Users\wnsgo\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\wnsgo\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\wnsgo\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

In [114]:
mag_s['ADFN_2'] = mag_s['ADFN_2'].str.replace("'DESIGN'", "디자인")

C:\Users\wnsgo\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [115]:
mag_s['ADFN_2'] = mag_s['ADFN_2'].str.replace("'피', '부과'", "피부과")
mag_s['ADFN_2'] = mag_s['ADFN_2'].str.replace("'패', '피'", "패피")
mag_s['ADFN_2'] = mag_s['ADFN_2'].str.replace("'유', '러', '피', '언'", "유러피언")



C:\Users\wnsgo\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\wnsgo\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\wnsgo\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [116]:
mag_s['ADFN_2'] = mag_s['ADFN_2'].str.replace("'밀레', '얼'", "밀레니얼")


C:\Users\wnsgo\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [117]:
mag_s['ADFN_2'] = mag_s['ADFN_2'].str.replace("'신', '제품'", "신제품")

C:\Users\wnsgo\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [118]:
sw_df = pd.read_csv('stopwords.csv', header=None)

In [119]:
sw_df.columns = ['stopword']

In [120]:
stop_words = []
for sw in sw_df['stopword']:
    stop_words.append(sw)

In [121]:
print(len(stop_words))
print(type(stop_words))

675
<class 'list'>


In [122]:
stop_words[-10:]

['둘', '셋', '넷', '다섯', '여섯', '일곱', '여덟', '아홉', '령', '영']

In [123]:
add_0 = ['진짜', '존나', '보고', '그냥', '카이', '이광수', '광수', '런닝맨', '❤️', '없는', '유재석', '❤', '♥', '있고', '거', 
         '내', '더', '개', '사람', '영상', '광고', '언니', '말', '뭐', '수', '생각',
        '있는', '정말', '같은', '같음', '듯', '♡', '완전', '같아', '정도', '지금', '역시', '뭔가', '입니다', '때문', '아니라',
         '함', '같은데', '해', '하는', '할', '넘', '한', '안', '했다', '보면', '데', '하고', '하면', '게', '해서', '봐도', 
         '한다', '된', '건', '요', '움', '임', '애', '하는거', '해도', '보는', '이제', '했는데', '하는데', '하는게', '같아요', 
         '합니다', '하지', '하게', '되는', 'I', '는', '난', '서', '도', '줄', '위', '분', '알', '은', '두', '번', '뒤',
         '이건', '없다', '있어요', '아니고', '있어', '링',
        '계속', '원래', '뭘', '못', '후', '이름', '속', '욕', '옆', '하네', '아님', '그런', '된다', '있음',
        '같', '돼', 'you', '제품', '표현', 'n', '봤는데', '건가', '나오는', '가지', '저런', '보이', '고',
        '중', '사진', '댓글', '부분', '진심', '그녀', '보', '날', '겁나', '요즘', '앞', '워', '이유', '솔직히', '하며', '통해',
        '모든', '항상', '전', '기자', '다시', '오늘', '↑', '이자', '대해', '', '이후', 'VALENTINO', '이렇', '이전', '유재', '만큼']
for w in add_0:
    stop_words.append(w)

In [124]:
len(stop_words)

829

In [125]:
mag_s['ADFN_2'] = mag_s['ADFN_2'].str.replace('[', '').str.replace(']', '').str.replace('"', '').str.replace("'", "").str.split(', ')

C:\Users\wnsgo\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [126]:
noun_adj_list = []
for i in range(len(mag_s)):
    for word in mag_s.loc[i, 'ADFN_2']:
        if word not in stop_words:
            noun_adj_list.append(word)

In [127]:
word = Counter(noun_adj_list)
print(word.most_common(200))

[('LAURENT', 2302), ('패션', 690), ('컬렉션', 559), ('브랜드', 532), ('디자이너', 420), ('ANTHONY', 371), ('쇼', 370), ('HEDI', 348), ('드레스', 340), ('소재', 316), ('스타일', 316), ('모델', 298), ('여성', 288), ('파리', 284), ('시즌', 277), ('팬츠', 269), ('장식', 254), ('디자인', 244), ('재킷', 234), ('하우스', 214), ('컬러', 204), ('옷', 187), ('가죽', 186), ('부츠', 178), ('그룹', 176), ('디렉터', 176), ('새롭', 162), ('블랙', 154), ('아이템', 154), ('슈트', 153), ('프랑스', 144), ('럭셔리', 141), ('CREATIVE', 133), ('COUTURE', 131), ('코트', 130), ('데님', 129), ('로고', 128), ('셔츠', 124), ('여름', 122), ('영감', 121), ('슈즈', 119), ('봄', 118), ('좋', 116), ('선글라스', 114), ('세계', 114), ('캠페인', 114), ('실루엣', 113), ('영화', 109), ('새', 108), ('시대', 105), ('어깨', 104), ('전시', 104), ('사랑', 101), ('톱', 99), ('시절', 97), ('라인', 96), ('수트', 96), ('미니', 95), ('패션계', 95), ('뷰티', 94), ('남성복', 94), ('즐기', 93), ('하이', 90), ('스커트', 90), ('완성', 90), ('트렌드', 90), ('시크', 90), ('패턴', 89), ('의상', 88), ('자리', 86), ('열리', 86), ('숄더', 85), ('분위기', 85), ('클래식', 84), ('무드', 84), ('파워',

In [128]:
count = Counter(noun_adj_list)

In [129]:
word_list = count.most_common(500)
for v in word_list:
    print(v)

('LAURENT', 2302)
('패션', 690)
('컬렉션', 559)
('브랜드', 532)
('디자이너', 420)
('ANTHONY', 371)
('쇼', 370)
('HEDI', 348)
('드레스', 340)
('소재', 316)
('스타일', 316)
('모델', 298)
('여성', 288)
('파리', 284)
('시즌', 277)
('팬츠', 269)
('장식', 254)
('디자인', 244)
('재킷', 234)
('하우스', 214)
('컬러', 204)
('옷', 187)
('가죽', 186)
('부츠', 178)
('그룹', 176)
('디렉터', 176)
('새롭', 162)
('블랙', 154)
('아이템', 154)
('슈트', 153)
('프랑스', 144)
('럭셔리', 141)
('CREATIVE', 133)
('COUTURE', 131)
('코트', 130)
('데님', 129)
('로고', 128)
('셔츠', 124)
('여름', 122)
('영감', 121)
('슈즈', 119)
('봄', 118)
('좋', 116)
('선글라스', 114)
('세계', 114)
('캠페인', 114)
('실루엣', 113)
('영화', 109)
('새', 108)
('시대', 105)
('어깨', 104)
('전시', 104)
('사랑', 101)
('톱', 99)
('시절', 97)
('라인', 96)
('수트', 96)
('미니', 95)
('패션계', 95)
('뷰티', 94)
('남성복', 94)
('즐기', 93)
('하이', 90)
('스커트', 90)
('완성', 90)
('트렌드', 90)
('시크', 90)
('패턴', 89)
('의상', 88)
('자리', 86)
('열리', 86)
('숄더', 85)
('분위기', 85)
('클래식', 84)
('무드', 84)
('파워', 84)
('패션쇼', 84)
('화이트', 82)
('완벽', 82)
('작품', 81)
('이미지', 81)
('백', 79)
('다

In [130]:
mag_s.to_csv('./magazine/final/magazine_s_stickword.csv', sep='|', encoding ='utf-8', index=False)